# IMPTOX 

## Random Forest modelling and Rules extraction with FIDEX (TBC)


For this example, we will consider that you have a fully numerical dataset without any missing values. 

This means that all values : 
- TODO COMPLETE CONDITIONS


Normalize data 0-1 ? 


In [1]:
import numpy as np
import pandas as pd
import pathlib as pl

import kagglehub

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

from dimlpfidex import fidex

# We'll try with this: https://hes-xplain.github.io/documentation/dimlpfidex/training-methods/randforeststrn/#arguments-list

from trainings import randForestsTrn # Part of DIMLPFIDEX



In [2]:
!pwd

/mnt/c/Users/TSchowing/Desktop/repositories/IMPTOX_XAI/notebooks


In [4]:
# Download latest version
path = kagglehub.dataset_download("gargmanas/pima-indians-diabetes")

print("Path to dataset files:", path )

df = pd.read_csv(path + '/pima-indians-diabetes.csv')
df.columns = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age','OUT']

X = df.drop('OUT', axis=1)
y = df['OUT']




# Shuffle and split the dataset into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42, shuffle=True, stratify=df['OUT'])

# Separate features and outputs for training and testing sets
train_features = train_data.drop(columns=['OUT'])
train_output = train_data['OUT']

test_features = test_data.drop(columns=['OUT'])
test_output = test_data['OUT']

# Save to CSV files
train_features.to_csv("./rf/train.csv", index=False)
train_output.to_csv("./rf/train_output.csv", index=False, header=True)

test_features.to_csv("./rf/test.csv", index=False)
test_output.to_csv("./rf/test_output.csv", index=False, header=True)

Path to dataset files: /home/twg/.cache/kagglehub/datasets/gargmanas/pima-indians-diabetes/versions/1


In [5]:
# Get df['OUT'] class imbalance
print(df['OUT'].value_counts(normalize=True))

OUT
0    0.65189
1    0.34811
Name: proportion, dtype: float64


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 767 entries, 0 to 766
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               767 non-null    int64  
 1   Glucose                   767 non-null    int64  
 2   BloodPressure             767 non-null    int64  
 3   SkinThickness             767 non-null    int64  
 4   Insulin                   767 non-null    int64  
 5   BMI                       767 non-null    float64
 6   DiabetesPedigreeFunction  767 non-null    float64
 7   Age                       767 non-null    int64  
 8   OUT                       767 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


# HES-XPLAIN RF thingy



In [15]:
!cat ../dimlpgui/config_wsl.json
#!pwd

{
  "root_folder": "/mnt/c/Users/TSchowing/Desktop/repositories/IMPTOX_XAI",
  "train_data_file": "/mnt/c/Users/TSchowing/Desktop/repositories/IMPTOX_XAI/notebooks/rf/train.csv",
  "train_class_file": "/mnt/c/Users/TSchowing/Desktop/repositories/IMPTOX_XAI/notebooks/rf/train_output.csv",
  "test_data_file": "/mnt/c/Users/TSchowing/Desktop/repositories/IMPTOX_XAI/notebooks/rf/test.csv",
  "test_class_file": "/mnt/c/Users/TSchowing/Desktop/repositories/IMPTOX_XAI/notebooks/rf/test_output.csv",
  "nb_attributes": 8,
  "nb_classes": 2,
  "train_pred_outfile": "predTrain.out",
  "test_pred_outfile": "predTest.out",
  "stats_file": "stats.txt",
  "console_file": "logs.txt",
  "rules_outfile": "rules.txt",
  "n_estimators": 100,
  "criterion": "gini",
  "max_depth": 5,
  "min_samples_split": 2,
  "min_samples_leaf": 1,
  "min_weight_fraction_leaf": 0.0,
  "max_features": "sqrt",
  "max_leaf_nodes": 5,
  "min_impurity_decrease": 0.0,
  "bootstrap": true,
  "oob_score": false,
  "n_jobs": 1,
  

In [16]:
randForestsTrn(
"""--json-configuration-file dimlpgui/config.json"""
)

Usage: 
--train_data_file <str> --test_data_file <str> --nb_attributes <int [1,inf[> --nb_classes <int [1,inf[> [-h, --help] [--json_config_file <str>] [--root_folder <str>] [--train_class_file <str>] [--test_class_file <str>] [--train_pred_outfile <str>] [--test_pred_outfile <str>] [--stats_file <str>] [--console_file <str>] [--rules_outfile <str>] [--n_estimators <int [1,inf[>] [--criterion <{gini, entropy, log_loss}>] [--max_depth <int [1,inf[>] [--min_samples_split <int [2,inf[ U float]0,1.0]>] [--min_samples_leaf <int [1,inf[ U float]0,1[>] [--min_weight_fraction_leaf <float [0,0.5]>] [--max_features <{sqrt, log2, all, float ]0,1[, int [1,inf[}>] [--max_leaf_nodes <int [2,inf[>] [--min_impurity_decrease <float [0,inf[>] [--bootstrap <bool>] [--oob_score <bool>] [--n_jobs <int>] [--seed <{int [0,inf[}>] [--verbose <int [0,inf[>] [--warm_start <bool>] [--class_weight <{balanced, balanced_subsample, dict}>] [--ccp_alpha <float [0,inf[>] [--max_samples <int [1,inf[ U float]0,1.0]>]
ip

-1

# Classic sklearn RF with grid search and CV


# Grid search



In [13]:
model = RandomForestClassifier()

RFgrid = {
    "n_estimators": [50, 100, 200],          # 3 options
    "max_depth": [10, 20, None],             # 3 options
    "min_samples_split": [2, 5, 10],         # 3 options
    "min_samples_leaf": [1, 2, 4],           # 3 options
    "bootstrap": [True, False]               # 2 options
}

# Repeated Stratified K-Fold (n_splits=4, controlled by n_repeats)
rskf = RepeatedStratifiedKFold(
    n_splits=5, 
    n_repeats=10, 
    random_state=42  # You can change this seed if desired
)

gs = GridSearchCV(
        estimator=model,
        param_grid=RFgrid,
        cv=rskf,
        n_jobs=2,
        scoring='accuracy',
        verbose=3,
        return_train_score=True
    )

# Fit the grid search
#gs.fit(X, y) # Done-> check the file bellow

# Export results to a DataFrame
#results = pd.DataFrame(gs.cv_results_)
#results.to_csv("./results_RF.csv", index=False)

# Print best parameters and score
#print("Best Parameters:", gs.best_params_)
#print("Best Score:", gs.best_score_)

Fitting 50 folds for each of 162 candidates, totalling 8100 fits
[CV 1/50] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=(train=0.989, test=0.753) total time=   0.1s
[CV 3/50] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=(train=0.992, test=0.771) total time=   0.0s
[CV 5/50] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=(train=0.997, test=0.784) total time=   0.0s
[CV 7/50] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=(train=0.993, test=0.708) total time=   0.0s
[CV 9/50] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=(train=0.990, test=0.745) total time=   0.0s
[CV 11/50] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=(train=0.992, test=0.773) total time=   0.0s
[CV 13/50]

In [14]:
# Print best parameters and score
print("Best Parameters:", gs.best_params_)
print("Best Score:", gs.best_score_)

Best Parameters: {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
Best Score: 0.7670206264323912

[CV 1/50] END bootstrap=False, max_depth=None, min_samples_leaf=4, min_samples_split=10, n_estimators=200;, score=(train=0.961, test=0.766) total time=   0.2s
[CV 2/50] END bootstrap=False, max_depth=None, min_samples_leaf=4, min_samples_split=10, n_estimators=200;, score=(train=0.966, test=0.753) total time=   0.2s
[CV 7/50] END bootstrap=False, max_depth=None, min_samples_leaf=4, min_samples_split=10, n_estimators=200;, score=(train=0.962, test=0.701) total time=   0.2s
[CV 8/50] END bootstrap=False, max_depth=None, min_samples_leaf=4, min_samples_split=10, n_estimators=200;, score=(train=0.972, test=0.765) total time=   0.1s
[CV 9/50] END bootstrap=False, max_depth=None, min_samples_leaf=4, min_samples_split=10, n_estimators=200;, score=(train=0.972, test=0.752) total time=   0.1s
[CV 10/50] END bootstrap=False, max_depth=None, mi


[CV 18/50] END bootstrap=False, max_depth=None, min_samples_leaf=4, min_samples_split=10, n_estimators=200;, score=(train=0.964, test=0.758) total time=   0.1s
[CV 23/50] END bootstrap=False, max_depth=None, min_samples_leaf=4, min_samples_split=10, n_estimators=200;, score=(train=0.969, test=0.804) total time=   0.1s
[CV 24/50] END bootstrap=False, max_depth=None, min_samples_leaf=4, min_samples_split=10, n_estimators=200;, score=(train=0.971, test=0.771) total time=   0.1s
[CV 25/50] END bootstrap=False, max_depth=None, min_samples_leaf=4, min_samples_split=10, n_estimators=200;, score=(train=0.956, test=0.758) total time=   0.1s
[CV 26/50] END bootstrap=False, max_depth=None, min_samples_leaf=4, min_samples_split=10, n_estimators=200;, score=(train=0.964, test=0.721) total time=   0.1s
[CV 31/50] END bootstrap=False, max_depth=None, min_samples_leaf=4, min_samples_split=10, n_estimators=200;, score=(train=0.961, test=0.779) total time=   0.1s
[CV 32/50] END bootstrap=False, max_dep